In [1]:
import os
import json
import time
import math
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '/Users/kevin_gwdong/djangoproject/tensorflow/lstmPrediction_twocolumn')

from core.data_processor import DataLoader
from core.model import Model
import pandas as pd
import numpy as np
import gc
from pathlib import Path



def convert_datetime_to_checkpoint( realdata, datetime):
    # 这是针对于有header的打他frame？是这样吗？需要验证
    checkpoint = realdata.loc[realdata['Date'] == datetime].index[0]
    return checkpoint 

def convert_checkpoint_to_datetime( realdata, checkpoint):
    datetime = realdata.iloc[checkpoint]['Date']
    return datetime 


def normalise_windows(window_data, single_window=False):
    '''Normalise window with a base value of zero'''
    normalised_data = []
    window_data = [window_data] if single_window else window_data
    for window in window_data:
        normalised_window = []
        for col_i in range(window.shape[1]):
            normalised_col = [((float(p) / float(window[0, col_i])) - 1) for p in window[:, col_i]]
            normalised_window.append(normalised_col)
        normalised_window = np.array(normalised_window).T # reshape and transpose array back into original multidimensional format
        normalised_data.append(normalised_window)
    return np.array(normalised_data)

def predict_5_full(startDate, stock):
    gc.collect()
#     high:
    p1_high = predict(startDate, stock, "High", "3")
    p2_high = predict(startDate, stock, "High", "4")
    p3_high = predict(startDate, stock, "High", "5")
    p4_high = predict(startDate, stock, "High", "6")
    p5_high = predict(startDate, stock, "High", "7")
#     low:
    p1_low = predict(startDate, stock, "Low", "3")
    p2_low = predict(startDate, stock, "Low", "4")
    p3_low = predict(startDate, stock, "Low", "5")
    p4_low = predict(startDate, stock, "Low", "6")
    p5_low = predict(startDate, stock, "Low", "7")
    
    p_5_h = [p1_high, p2_high, p3_high, p4_high, p5_high]
    p_5_l = [p1_low, p2_low, p3_low, p4_low, p5_low]
    gc.collect()
    realdata = pd.read_csv(Path('data/' + stock + '.CSV').resolve())
    startPoint = convert_datetime_to_checkpoint(realdata, startDate)
    
    if realdata.iloc[-1]['Date'] == startDate:
        date = "tomorrow"
    else:
        date = realdata.iloc[startPoint+1]['Date']
    print(date)   
    predictions = {
        "code":stock,
        "date":date,
        "high":p_5_h,
        "low":p_5_l
    }
    # file_to_save = "../predictions_result/" + stock + "_" + column + "_" + startDate + ".json"
    file_to_save = Path("predictions_result_volume/stock/" + stock + "_" + startDate + ".json").resolve()
    with open(file_to_save, 'w') as f:
        json.dump(predictions, f)
    print(predictions)
    gc.collect()
    return predictions

'''
    {
        "code":'000738',
        'date': '2019-09-10',
        "high":[13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382],
        "low":[13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382]
    }

'''    
def predict(startDate, stock, column, sequence_length ) :
    # configs = json.load(open('../configs/config.json', 'r'))
    # global configs
    gc.collect()
    configs = json.load(open(Path('configs/config.json').resolve(),'r'))

    if column == "Close" :
        configs['data']['columns'] = ["Close", "Volume"]
    elif column == "High" :
        configs['data']['columns'] = ["High",  "Volume"]
    elif column == "Low" :
        configs['data']['columns'] = ["Low", "Volume"]

    configs['data']['filename'] = stock + ".csv"
    configs['data']['sequence_length'] = int(sequence_length)
    configs['model']['layers'][0]['input_timesteps'] = int(sequence_length) - 1


    # model_save_dir = "../bestmodels/" + sequence_length + "/" + stock + "-" + column + ".h5"
    model_save_dir = Path("bestmodels/volume/" + sequence_length + "/" + stock + "-" + column + ".h5").resolve()
    # print(model_save_dir)
    model = Model()
    model.load_model(str(model_save_dir))

    # realdata = pd.read_csv('data/' + stock + '.CSV')
    # global realdata
    realdata = pd.read_csv(Path('data/' + stock + '.CSV').resolve())

    startPoint = convert_datetime_to_checkpoint(realdata, startDate)+1

    raw_data_windows = realdata.get(configs['data']['columns']).values[startPoint-int(sequence_length)+1:startPoint]

    data_windows = np.array(raw_data_windows).astype(float)
#     print(data_windows)
    data_windows = normalise_windows(data_windows, single_window=True)
    prediction = model.predict_point_by_point(data_windows)
    prediction = raw_data_windows[0][0]* (1 + prediction[0])
    gc.collect()
    return prediction



Using TensorFlow backend.


### 有close的预测

In [1]:
import os
import json
import time
import math
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '/Users/kevin_gwdong/djangoproject/tensorflow/lstmPrediction_twocolumn')

from core.data_processor import DataLoader
from core.model import Model
import pandas as pd
import numpy as np
import gc
from pathlib import Path
from pymongo import MongoClient

def convert_datetime_to_checkpoint( realdata, datetime):
    # 这是针对于有header的打他frame？是这样吗？需要验证
    checkpoint = realdata.loc[realdata['Date'] == datetime].index[0]
    return checkpoint 

def convert_checkpoint_to_datetime( realdata, checkpoint):
    datetime = realdata.iloc[checkpoint]['Date']
    return datetime 


def normalise_windows(window_data, single_window=False):
    '''Normalise window with a base value of zero'''
    normalised_data = []
    window_data = [window_data] if single_window else window_data
    for window in window_data:
        normalised_window = []
        for col_i in range(window.shape[1]):
            normalised_col = [((float(p) / float(window[0, col_i])) - 1) for p in window[:, col_i]]
            normalised_window.append(normalised_col)
        normalised_window = np.array(normalised_window).T # reshape and transpose array back into original multidimensional format
        normalised_data.append(normalised_window)
    return np.array(normalised_data)

def predict_5_full(startDate, stock, conn):
    gc.collect()
    
    predictions = {}    
    predictions['_id'] = stock + "_" + startDate
    db = conn.stock

    realdata = pd.read_csv(Path('data/' + stock + '.CSV').resolve())
    startPoint = convert_datetime_to_checkpoint(realdata, startDate)
    
    if realdata.iloc[-1]['Date'] == startDate:
        date = "tomorrow"
    else:
        date = realdata.iloc[startPoint+1]['Date']
    print(date)       
    
    if stock in db.list_collection_names() :
        print(stock, " already have, check if high low close exist")
#         try:
        query = { "_id": predictions['_id'] }
        record = db[str(stock)].find_one(query)
        if record != None :
            print(stock, " we have high and low, test if we have close")
            if "close" in record :
                print(stock, " we have close, we have everything, can update")
                newvalues = {"$set":{
                "code":stock,
                "date":date,
                "high":record['high'],
                "low":record['low'],
                "close":record['close']
                }}
                db[str(stock)].update_one(query, newvalues)
            else:
                print(stock, " we dont have close")
            # we dont have close, do close work.
            #     close:
                p1_close = predict(startDate, stock, "Close", "3")
                p2_close = predict(startDate, stock, "Close", "4")
                p3_close = predict(startDate, stock, "Close", "5")
                p4_close = predict(startDate, stock, "Close", "6")
                p5_close = predict(startDate, stock, "Close", "7")
                p_5_c = [p1_close, p2_close, p3_close, p4_close, p5_close]
                newvalues = {"$set":{
                "code":stock,
                "date":date,
                "high":record['high'],
                "low":record['low'],
                "close":p_5_c
                }}
                db[str(stock)].update_one(query, newvalues)
        else:
            print(stock, " this day's record not exist we try to predict all")                
        #     high:
            p1_high = predict(startDate, stock, "High", "3")
            p2_high = predict(startDate, stock, "High", "4")
            p3_high = predict(startDate, stock, "High", "5")
            p4_high = predict(startDate, stock, "High", "6")
            p5_high = predict(startDate, stock, "High", "7")
        #     low:
            p1_low = predict(startDate, stock, "Low", "3")
            p2_low = predict(startDate, stock, "Low", "4")
            p3_low = predict(startDate, stock, "Low", "5")
            p4_low = predict(startDate, stock, "Low", "6")
            p5_low = predict(startDate, stock, "Low", "7")
        #     close:
            p1_close = predict(startDate, stock, "Close", "3")
            p2_close = predict(startDate, stock, "Close", "4")
            p3_close = predict(startDate, stock, "Close", "5")
            p4_close = predict(startDate, stock, "Close", "6")
            p5_close = predict(startDate, stock, "Close", "7")

            p_5_h = [p1_high, p2_high, p3_high, p4_high, p5_high]
            p_5_l = [p1_low, p2_low, p3_low, p4_low, p5_low]
            p_5_c = [p1_close, p2_close, p3_close, p4_close, p5_close]

            predictions = {
                '_id':stock + "_" + startDate,
                "code":stock,
                "date":date,
                "high":p_5_h,
                "low":p_5_l,
                "close":p_5_c
            }        
            db[str(stock)].insert_one(predictions)
#         except:
#             print("something wrong happen!")
    else:
        print(stock, " we dont have collection, creat one and predict")
        db.create_collection(stock)            
    #     high:
        p1_high = predict(startDate, stock, "High", "3")
        p2_high = predict(startDate, stock, "High", "4")
        p3_high = predict(startDate, stock, "High", "5")
        p4_high = predict(startDate, stock, "High", "6")
        p5_high = predict(startDate, stock, "High", "7")
    #     low:
        p1_low = predict(startDate, stock, "Low", "3")
        p2_low = predict(startDate, stock, "Low", "4")
        p3_low = predict(startDate, stock, "Low", "5")
        p4_low = predict(startDate, stock, "Low", "6")
        p5_low = predict(startDate, stock, "Low", "7")
    #     close:
        p1_close = predict(startDate, stock, "Close", "3")
        p2_close = predict(startDate, stock, "Close", "4")
        p3_close = predict(startDate, stock, "Close", "5")
        p4_close = predict(startDate, stock, "Close", "6")
        p5_close = predict(startDate, stock, "Close", "7")

        p_5_h = [p1_high, p2_high, p3_high, p4_high, p5_high]
        p_5_l = [p1_low, p2_low, p3_low, p4_low, p5_low]
        p_5_c = [p1_close, p2_close, p3_close, p4_close, p5_close]

        predictions = {
            "_id":stock + "_" + startDate,
            "code":stock,
            "date":date,
            "high":p_5_h,
            "low":p_5_l,
            "close":p_5_c
        }         
        db[str(stock)].insert_one(predictions)
        
        
    query = { "_id": predictions['_id'] }
    predictions = db[str(stock)].find_one(query)
    print("currently doing: ",predictions['_id'])
    print(predictions)    
    print()
    
    return predictions
  
def predict(startDate, stock, column, sequence_length ) :
    # configs = json.load(open('../configs/config.json', 'r'))
    # global configs
    gc.collect()
    configs = json.load(open(Path('configs/config.json').resolve(),'r'))

    if column == "Close" :
        configs['data']['columns'] = ["Close", "Volume"]
    elif column == "High" :
        configs['data']['columns'] = ["High",  "Volume"]
    elif column == "Low" :
        configs['data']['columns'] = ["Low", "Volume"]

    configs['data']['filename'] = stock + ".csv"
    configs['data']['sequence_length'] = int(sequence_length)
    configs['model']['layers'][0]['input_timesteps'] = int(sequence_length) - 1


    # model_save_dir = "../bestmodels/" + sequence_length + "/" + stock + "-" + column + ".h5"
    model_save_dir = Path("bestmodels/volume/" + sequence_length + "/" + stock + "-" + column + ".h5").resolve()
    # print(model_save_dir)
    model = Model()
    model.load_model(str(model_save_dir))

    # realdata = pd.read_csv('data/' + stock + '.CSV')
    # global realdata
    realdata = pd.read_csv(Path('data/' + stock + '.CSV').resolve())

    startPoint = convert_datetime_to_checkpoint(realdata, startDate)+1

    raw_data_windows = realdata.get(configs['data']['columns']).values[startPoint-int(sequence_length)+1:startPoint]

    data_windows = np.array(raw_data_windows).astype(float)
#     print(data_windows)
    data_windows = normalise_windows(data_windows, single_window=True)
    prediction = model.predict_point_by_point(data_windows)
    prediction = raw_data_windows[0][0]* (1 + prediction[0])
    gc.collect()
    return prediction



def main():
    parser = argparse.ArgumentParser()

#     parser.add_argument("--predict_one", "-po", help="start date")
    parser.add_argument("--start_date", "-sd", help="start date")
#     parser.add_argument("--end_date", "-ed", help="end date")
    parser.add_argument("--stockcode", "-sc", help="stock code name")

    args = parser.parse_args()

    try: 
        conn = MongoClient() 
        print("Connected successfully!!!") 
    except:   
        print("Could not connect to MongoDB") 

    startDate = args.start_date
    stock = args.stockcode
    predict_5_full(startDate, stock, conn)
    conn.close()
    
    # print(stock)
#     realdata = pd.read_csv(Path('data/' + stock + '.CSV').resolve())
    
#     if args.predict_one == "true":
#         print(startDate)
#         id= stock + "_" + date 
#         db = conn.stock
#         query = { "_id": id }
#         data = db[str(stock)].find_one(query)
#         if data == None :
#             predict_5_full(date, stock, conn)
#             print(date," we don't have this data, start predict")
#         elif data['date'] == "tomorrow" :
#             print(date, " data is tomorrow, we need to re-run this")
#             predict_5_full(date, stock, conn)
#         else:
#             print(date, "we have this data, no need to do again")

#         # predict_5_full(startDate, stock, conn)
#     else:
#         enddate = args.end_date
#         data_selected = realdata.loc[(realdata['Date'] >= startDate) & (realdata['Date'] <= enddate)]['Date'].values
#         print(data_selected)
        
#         for date in data_selected:
#             gc.collect()
#             id= stock + "_" + date 
#             db = conn.stock
#             query = { "_id": id }
#             data = db[str(stock)].find_one(query)
#             if data == None :
#                 print(date, " we don't have this data, start predict")
#                 predict_5_full(date, stock, conn)
#             elif data['date'] == "tomorrow" :
#                 print(date, " data is tomorrow, we need to re-run this")
#                 predict_5_full(date, stock, conn)
#             else:
#                 print(date, " we have this data, no need to do again")
# if __name__ == '__main__':
#     main()


# try: 
#     conn = MongoClient() 
#     print("Connected successfully!!!") 
# except:   
#     print("Could not connect to MongoDB")  

# stock = '600150'
# startDate = '2019-09-19'
# predict_5_full(startDate, stock, conn)



Using TensorFlow backend.


Connected successfully!!!
2019-09-20
600150  already have, check if high low close exist
600150  we have high and low, test if we have close
600150  we have close, we have everything, can update
currently doing:  600150_2019-09-19
{'_id': '600150_2019-09-19', 'code': '600150', 'date': '2019-09-20', 'high': [24.44427627597004, 24.595371733233332, 24.19162942968309, 25.429712792634966, 24.639821371547875], 'low': [23.753893183479086, 24.453880053414032, 23.801147470772268, 24.41021479714662, 24.144074062779545], 'close': [24.260704134383705, 24.255610937383025, 24.598400983959436, 25.556029208004475, 24.520337117195595]}


{'_id': '600150_2019-09-19',
 'code': '600150',
 'date': '2019-09-20',
 'high': [24.44427627597004,
  24.595371733233332,
  24.19162942968309,
  25.429712792634966,
  24.639821371547875],
 'low': [23.753893183479086,
  24.453880053414032,
  23.801147470772268,
  24.41021479714662,
  24.144074062779545],
 'close': [24.260704134383705,
  24.255610937383025,
  24.598400983959436,
  25.556029208004475,
  24.520337117195595]}

In [13]:
# 
# startDate = '2019-09-19'
stock = '600150' # 中国船舶


# predict(startDate, stock, column, sequence_length )
realdata = pd.read_csv('data/' + stock + '.CSV')
data_selected = realdata.loc[(realdata['Date'] > '2019-12-09') & (realdata['Date'] < '2019-12-20')]['Date'].values
print(data_selected)

# predict_5_full('2019-09-18', stock)
# code_list = []

# for date in data_selected:
# #     gc.collect()
# #     predict_5_full(date, stock)
#     code_list.append(code)

# try: 
#     conn = MongoClient() 
#     print("Connected successfully!!!") 
# except:   
#     print("Could not connect to MongoDB")     
    
new_list = [data_selected[i*4:i*4+4] for i in range(int(len(data_selected)/4+1))]
# new_list = [[603721, 603718, 603665, 603578], [603229, 603108, 601698, 600862], [600720, 600703, 600671, 600660], [600637, 600597, 600377, 600298], [600196, 600079, 600060, 600004]]
print(new_list)
time_init = 300
for group in new_list:
    for date in group :
        print("now we are doing this",date)
        predict_5_full_close(date, stock, conn)
    time.sleep(time_init)
    time_init= time_init+120
    print()     
    
    

['2019-12-10' '2019-12-11' '2019-12-12' '2019-12-13' '2019-12-16'
 '2019-12-17' '2019-12-18' '2019-12-19']
[array(['2019-12-10', '2019-12-11', '2019-12-12', '2019-12-13'],
      dtype=object), array(['2019-12-16', '2019-12-17', '2019-12-18', '2019-12-19'],
      dtype=object), array([], dtype=object)]
now we are doing this 2019-12-10
2019-12-11
{'code': '600150', 'date': '2019-12-11', 'high': [20.808384922025727, 20.867798846345394, 20.56043630269356, 20.56492784384638, 21.07589416205883], 'low': [20.37528644882841, 20.071784756274425, 20.318304409720003, 19.801973994262514, 20.546668966300786], 'close': [20.674021056685596, 20.509540706034752, 20.598980691656468, 20.7365649786219, 20.389949499815703]}
{'code': '600150', 'date': '2019-12-11', 'high': [20.808384922025727, 20.867798846345394, 20.56043630269356, 20.56492784384638, 21.07589416205883], 'low': [20.37528644882841, 20.071784756274425, 20.318304409720003, 19.801973994262514, 20.546668966300786], 'close': [20.674021056685596, 20

NameError: name 'conn' is not defined

In [2]:
import subprocess
import shlex
subprocess.call(shlex.split('./run_close.sh 600150'))
subprocess.call(shlex.split('./run_close.sh 002609'))

0

In [2]:
# 
# startDate = '2019-09-19'
stock = '600150'


# predict(startDate, stock, column, sequence_length )
realdata = pd.read_csv('data/' + stock + '.CSV')
data_selected = realdata.loc[(realdata['Date'] > '2019-12-09') & (realdata['Date'] < '2019-12-20')]['Date'].values
print(data_selected)

# predict_5_full('2019-09-18', stock)

for date in data_selected:
    gc.collect()
    predict_5_full(date, stock)
    




['2019-12-10' '2019-12-11' '2019-12-12' '2019-12-13' '2019-12-16'
 '2019-12-17' '2019-12-18' '2019-12-19']
2019-12-11
{'code': '600273', 'date': '2019-12-11', 'high': [9.881097362190484, 9.902788590397686, 9.874099856987597, 9.964093058109283, 9.894068744732067], 'low': [9.693595439773054, 9.698658611699939, 9.720417498052122, 9.662598345130682, 9.70999858884141]}
2019-12-12
{'code': '600273', 'date': '2019-12-12', 'high': [10.156606001257897, 10.211906186118721, 10.150695068649947, 10.103415874298662, 10.129572808295489], 'low': [9.857550792843105, 9.875090418923646, 9.864542255736888, 9.818207073770465, 9.772461238410324]}
2019-12-13
{'code': '600273', 'date': '2019-12-13', 'high': [10.079748021438718, 10.056038220226764, 10.079683845117687, 10.031971247456967, 10.03592391544953], 'low': [9.873303701682017, 9.873474586978555, 9.877760065728799, 9.827440265826882, 9.83272573715076]}
2019-12-16
{'code': '600273', 'date': '2019-12-16', 'high': [10.097876970358193, 10.114083824157714, 10

In [2]:
stock = '600273'

predict_5_full('2019-12-09', stock)
# predict_5_full('2019-12-19', stock)
predict_5_full('2019-12-20', stock)

predict_5_full('2019-12-26', stock)


2019-12-10
{'code': '600273', 'date': '2019-12-10', 'high': [10.23231404332444, 10.218491841536016, 10.206399797201156, 10.233278625011444, 10.158531320095063], 'low': [9.837769505884498, 9.841812307015061, 9.805731458580121, 9.73402933228761, 9.787616497278213]}
2019-12-23
{'code': '600273', 'date': '2019-12-23', 'high': [10.736831674352288, 10.795922256819903, 10.684115947186948, 10.7428860925138, 10.652130166143179], 'low': [10.382286494411527, 10.370398411955684, 10.380559927076101, 10.312631696537137, 10.387824980542064]}
2019-12-27
{'code': '600273', 'date': '2019-12-27', 'high': [11.989959039613604, 11.821040415763855, 11.888188874050975, 11.543336151838302, 11.715429595112802], 'low': [10.935366724431516, 10.922169032692908, 10.951446157693862, 10.835131924003363, 10.871518766432999]}


{'code': '600273',
 'date': '2019-12-27',
 'high': [11.989959039613604,
  11.821040415763855,
  11.888188874050975,
  11.543336151838302,
  11.715429595112802],
 'low': [10.935366724431516,
  10.922169032692908,
  10.951446157693862,
  10.835131924003363,
  10.871518766432999]}

In [3]:
stock = '603998'

# predict_5_full('2019-12-30', stock)
# predict_5_full('2020-01-06', stock)
predict_5_full('2019-11-20', stock)

OSError: Unable to open file (unable to open file: name = '/Users/kevin_gwdong/djangoproject/tensorflow/lstmPrediction_twocolumn/bestmodels/volume/3/603998-High.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)